In [1]:
!pip install yfinance --quiet
import yfinance as yf
import pandas as pd
import numpy as np

# --- Risk free rate and storage ---
RISK_FREE_RATE = 0.05  # annual, continuous
STORAGE_COST_PER_TON_MONTH = 20  # example for steel
DAYS_IN_YEAR = 365

# --- Steel multipliers ---
material_multipliers = {
    'A36': 1.0,
    '4140': 1.25,
    '304SS': 2.5,
    'A572': 1.1,
    'Steel': 1.0,
    'Aluminum': 1.0,  # override with live if available
    'Titanium': 1.0   # fallback model
}

# --- Tickers ---
tickers = {
    'hrc': 'HRC=F',      # Hot Rolled Coil steel
    'scrap': 'HG=F',     # Copper futures as proxy
    'iron_ore': None,    # fallback
    'oil': 'CL=F',       # WTI crude
    'gas': 'NG=F',       # Natural Gas
    'aluminum': 'ALI=F'  # example ticker
}

avg_prices = {
    'hrc': 850,
    'scrap': 450,
    'iron_ore': 115,
    'oil': 80,
    'gas': 3,
    'aluminum': 2500
}

weights = {'hrc': 0.55, 'scrap': 0.15, 'iron_ore': 0.10, 'oil': 0.10, 'gas': 0.05, 'premium': 0.05}
BASE_A36 = 800

# --- Fetch price ---
def get_price(ticker, fallback):
    if ticker is None:
        return fallback
    try:
        data = yf.Ticker(ticker).history(period='1d')
        if data.empty:
            raise ValueError("No data")
        return float(data['Close'].iloc[-1])
    except:
        return fallback

# --- Weighted model for steel ---
def weighted_steel_price(material='A36'):
    prices = {k: get_price(tickers.get(k), avg_prices.get(k)) for k in tickers}
    base = BASE_A36 * (
        weights['hrc']*(prices['hrc']/avg_prices['hrc']) +
        weights['scrap']*(prices['scrap']/avg_prices['scrap']) +
        weights['iron_ore']*(prices['iron_ore']/avg_prices['iron_ore']) +
        weights['oil']*(prices['oil']/avg_prices['oil']) +
        weights['gas']*(prices['gas']/avg_prices['gas']) +
        weights['premium']
    )
    multiplier = material_multipliers.get(material.upper(),1.0)
    return round(base * multiplier,2), prices

# --- Futures 7-day projection ---
def futures_projection(spot_price, storage_per_ton=20, risk_free=0.05):
    u = (storage_per_ton*12) / spot_price  # fraction per year
    days = np.arange(1,8)  # 1 to 7
    projected = [spot_price * np.exp((risk_free + u) * d/DAYS_IN_YEAR) for d in days]
    return [round(p,2) for p in projected]

# --- Full function for material ---
def predict_material(material='A36'):
    material_upper = material.upper()
    if material_upper in ['A36','4140','304SS','A572','STEEL']:
        spot, inputs = weighted_steel_price(material_upper)
    else:  # Aluminum, Titanium
        spot = get_price(tickers.get(material.lower()), avg_prices.get(material.lower()))
        inputs = {material: spot}
    projection = futures_projection(spot)
    return spot, projection, inputs

# --- Example usage ---
material = 'A36'
spot, projection, inputs = predict_material(material)
print(f"Material: {material}")
print(f"Spot price today: ${spot}/ton")
print(f"7-day projected prices: {projection}")
print(f"Inputs used: {inputs}")


Material: A36
Spot price today: $640.33/ton
7-day projected prices: [np.float64(641.08), np.float64(641.82), np.float64(642.57), np.float64(643.32), np.float64(644.07), np.float64(644.82), np.float64(645.57)]
Inputs used: {'hrc': 814.0, 'scrap': 4.93149995803833, 'iron_ore': 115, 'oil': 57.540000915527344, 'gas': 3.007999897003174, 'aluminum': 2683.0}
